<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 2: Working with Text

这个笔记本中使用的包：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.0.1
tiktoken version: 0.9.0


本章涵盖了数据准备和采样，以使输入数据为LLM“准备就绪”

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 Understanding word embeddings

深度神经网络模型，包括 LLM，往往无法直接处理原始文本。这是因为文本是离散的分类数据，它与实现和训练神经网络所需的数学运算不兼容。因此，我们需要一种方法将单词表示为连续值向量，即嵌入（embeddings）——将数据转换为向量格式的过程。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

如上图所示，不同格式的数据源（如文本、图像、音频、视频）在处理和嵌入时，需要不同的模型和技术，原因在于它们的数据结构、特征和处理方式各不相同，因此需要针对性的方法将这些不同的数据类型转换为适合神经网络处理的向量表示。以下总结了不同数据源在嵌入时的一些区别：
| 数据类型 |          数据特征          |              嵌入模型              |          主要特征          |
| :------: | :------------------------: | :--------------------------------: | :------------------------: |
|   文本   |  离散的、序列化的符号数据  |   Word2Vec, GloVe, BERT, GPT 等    |    语义关系、上下文理解    |
|   图像   | 二维像素网格，具有空间特征 |      CNN（ResNet、VGG）、ViT       | 形状、纹理、颜色等视觉特征 |
|   音频   |        一维时序信号        |    CNN+频谱图、RNN、Transformer    |    频率、音调、时序依赖    |
|   视频   |        时空序列数据        | 3D CNN、RNN+CNN、Video Transformer |     时空特征、动作捕捉     |


嵌入的本质是将离散对象（如单词、图像或整个文档）映射到连续向量空间中的点。嵌入的主要目的是将非数值数据转换为神经网络能够处理的格式。在自然语言处理（NLP）中，嵌入通常用于将单词、短语或整个文档转换为向量表示。这些向量表示可以捕获单词之间的语义和上下文关系，从而使神经网络能够更好地理解文本数据。

一般来说，单词嵌入是LLM最常用的文本嵌入形式，句子、段落或整篇文档的嵌入常被应用于检索增强生成技术——结合了文本生成与从外部知识库中检索相关信息的过程。

词嵌入可以具有不同的维度，从一维到数千维，具体取决于所使用的模型和任务。例如，Word2Vec 模型通常使用 100 到 300 维的嵌入，而 BERT 模型使用 768 维的嵌入，LLM 则使用数千维的嵌入。更高的维度可能捕捉到更细微的关系，但代价是计算效率的降低。

为了可视化，下图展示了一个简化的例子，其中我们将单词映射到一个二维空间，可以看到相似的词汇聚在一起。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

本章接下来的部分将系统地介绍准备 LLM 使用的嵌入所需的步骤，这些步骤包括将文本拆分为单词、将单词转换为token，以及将token转化为嵌入向量。

## 2.2 文本分词

本节将讨论如何将输入文本拆分为单个token，这是创建 LLM 嵌入所需的预处理步骤。这些token可以是单个单词或特殊字符，包括标点符号，具体如图所示

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

我们即将用于 LLM 训练的文本数据集是一部由 Edith Wharton 创作的短篇小说《判决》

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是为LLM对这部文本进行token化和嵌入
- 让我们基于一些简单的示例文本开发一个简单的分词器，然后我们可以将其应用于上面的文本

In [4]:
import re

# 以下正则表达式将在空格上拆分
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


上述简单的分词方案仅仅用于将示例文本拆分为单个单词，然而有些单词仍然与我们希望单独列出的标点符号相连。

需要修改正则表达式，将空白字符（\s）、逗号和句点（[,.]）单独拆分出。此外，我们也无需将所有文本转换为小写字母，因为大写字母有助于 LLM 区分专有名词和普通名词，理解句子结构，并学习生成正确的大写文本。

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
# 删除空字符串
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


> 是否将空白字符编码为单独的字符，或者直接将其删除，取决于我们的应用和需求。
> 删除空白字符可以减少内存和计算资源的消耗。然而，如果我们训练的模型对文本的确切结构敏感（例如，Python 代码对缩进和空格非常敏感），那么保留空白字符就很有用。

进一步修改这个分词方案，使其能够处理其他类型的标点符号，如问号、引号等。

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


如图所示，目前实现的分词方案已经能够很好地将文本拆分为单个单词和标点符号。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
# 将分词方案应用到原始文本
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print("token数量:", len(preprocessed))

token数量: 4690


In [9]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## 2.3 将 tokens 转换为token IDs

在本节中，我们将讨论如何将 token 转换为 token ID，这是将文本数据转换为 LLM 可以处理的格式的下一步。在这个过程中，我们为每个 token 分配一个唯一的整数 ID，这样 LLM 就可以将这些 token ID 作为输入。这一步是将token转换为嵌入向量的前提。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

为了将先前生成的token映射到token ID，我们首先需要构建一个词汇表。这个词汇表定义了每个**独特单词和特殊字符**与**唯一整数**的映射

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [ ]:
# 创建一个词汇表，将单词映射到整数
vocab = {token:integer for integer,token in enumerate(all_words)}

下面是词汇表的前51个条目

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


词汇表将token转化为token ID，我们还需要一种将token ID 转化为 token 的方法，这样我们就可以将模型的输出转换为文本。为此，我们可以创建一个词汇表的反向版本，将token ID 映射回相应的文本token。

将上述所有方法整合到一个完整的分词器类中，这个类将负责将文本转换为token ID，（encode）以及将token ID 转换为文本（decode）。

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        """
        将文本分词并将其转换为整数token ID
        """
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        """
        将整数token ID转换为文本
        """
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

让我们通过 SimpleTokenizerV1 类实例化一个新的分词器对象，并对艾迪丝·华顿的短篇小说中的一段文本进行分词，以便在实践中进行尝试：

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


通过 decode 方法将这些token ID 转换回文本：

In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Adding special context tokens

为未知单词和文本结束添加特殊的上下文标记是很有用的。具体来说，我们将修改在前一节中实现的词汇表和分词器类（修改后的类命名为SimpleTokenizerV2），以支持两个新的token：<|unk|> 和 <|endoftext|>
- <|unk|> 用于表示未知的单词
- <|endoftext|> 用于表示文本的结束，这有助于大语言模型理解，尽管这些文本源在训练中是连接在一起的，但它们实际上是无关的。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

一些分词器使用特殊标记来帮助LLM获得额外的上下文
- 一些特殊标记如下
  - `[BOS]`（序列开始）标记文本的开始
  - `[EOS]`（序列结束）标记文本的结束（通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书等）
  - `[PAD]`（填充）如果我们使用大于1的批量大小训练LLM（我们可能包含多个长度不同的文本；使用填充标记，我们将较短的文本填充到最长长度，以便所有文本具有相同的长度）
  - -  对于 [PAD] 标记，我们通常使用掩码（mask）来忽略填充的标记，以便模型不会关注填充的标记.在输入批次时，我们为每个 token 位置创建一个掩码矩阵，用来标识哪些位置是有效 token（模型应该关注），哪些是填充 token（模型应该忽略）。假设 1 表示有效 token，0 表示填充 token，则掩码矩阵如下：
    
    句子1（掩码）：[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
    
    句子2（掩码）：[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
    
    句子3（掩码）：[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

  - `[UNK]` 用于表示词汇表中没有包含的单词

需要注意的是，GPT-2 不需要上述任何标记，只使用 `<|endoftext|>` 标记来降低复杂性
- `<|endoftext|>` 类似于上面提到的 `[EOS]` 标记
GPT 还使用 `<|endoftext|>` 进行填充（因为我们通常在批量输入训练时使用掩码，所以我们不会关注填充的标记，因此这些标记是什么并不重要）

GPT-2 不使用 `<UNK>` 标记来表示词汇表中没有的单词；相反，GPT-2 使用字节对编码（BPE）分词器，将单词分解为子词单元，我们将在后面的章节中讨论



<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

将原来的分词器应用到新的文本：

In [18]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

上面的代码会产生错误，因为单词“Hello”不在词汇表中

为了处理这种情况，我们可以将特殊标记如 `"<|unk|>"` 添加到词汇表中，以表示未知单词

由于我们已经在扩展词汇表，让我们添加另一个称为 `"<|endoftext|>"` 的标记，它在 GPT-2 训练中用于表示文本的结束（也用于连接的文本之间，例如，如果我们的训练数据集由多个文章、书籍等组成）

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


调整分词器类，以支持新的特殊标记：

In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

GPT-2 使用 BytePair 编码（BPE）作为其分词器。它允许模型将不在预定义词汇表中的单词分解为更小的子词单元，甚至是单个字符，从而使其能够处理词汇表之外的单词

例如，如果 GPT-2 的词汇表中没有单词“unfamiliarword”，它可能将其分词为 ["unfam", "iliar", "word"] 或其他一些子词分解，具体取决于其训练的 BPE 合并

原始的 BPE 分词器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)

在本章中，我们使用 OpenAI 的开源库 [tiktoken](https://github.com/openai/tiktoken) 

In [ ]:
# pip install tiktoken

In [25]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [26]:
tokenizer = tiktoken.get_encoding("gpt2")

In [27]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [28]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


BPE 分词器将不在其预定义词汇表中的单词分解为更小的子词单元甚至单个字符，使其能够处理超出词汇表的单词。因此，得益于BPE算法，如果分词器在分词过程中遇到一个不熟悉的单词，它可以将其表示为一系列子词token或字符

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

字节对编码是一种基于统计的方法，它会先从整个语料库中找出最常见的字节对（byte pair），然后把这些字节对合并成一个新的单元。让我们用一个具体的示例来描述这个过程：

假如有句子：“The cat drank the milk because it was hungry”

1. 初始化：BPE会先将句子中每个字符视为一个单独的token

```python
['T', 'h', 'e', ' ', 'c', 'a', 't', ' ', 'd', 'r', 'a', 'n', 'k', ' ', 't', 'h', 'e', ' ', 'm', 'i', 'l', 'k', ' ', 'b', 'e', 'c', 'a', 'u', 's', 'e', ' ', 'i', 't', ' ', 'w', 'a', 's', ' ', 'h', 'u', 'n', 'g', 'r', 'y']
```

2. 统计最常见的字节对

BPE算法会在这些token中找到出现频率最高的“字节对”（即相邻的两个字符），然后将其合并为一个新的token。

例如这里最常见的字节对时（'t', 'h'），因为它在单词"the"和"that"中出现频率较高。

3. 合并字节对

根据统计结果，我们将最常见的字节对（'t', 'h'）合并为一个新的token，其它类似

```python
['Th', 'e', ' ', 'c', 'a', 't', ' ', 'dr', 'a', 'nk', ' ', 'th', 'e', ' ', 'm', 'i', 'l', 'k', ' ', 'be', 'c', 'a', 'u', 'se', ' ', 'it', ' ', 'wa', 's', ' ', 'hu', 'n', 'gr', 'y']
```

4. 重复步骤2和3，得到最终的token序列

```python
['The', ' ', 'cat', ' ', 'drank', ' ', 'the', ' ', 'milk', ' ', 'because', ' ', 'it', ' ', 'was', ' ', 'hungry']
```

## 2.6 使用滑动窗口进行数据采样

我们训练LLM以每一次生成一个单词，因此我们希望相应地准备训练数据，其中序列中的下一个单词为要预测的目标。

在本节中，我们将实现一个数据加载器，通过滑动窗口方法从训练数据集中提取下图所示的输入-目标对。这种方法将文本数据集转换为模型可以处理的格式，其中输入是一个序列，目标是该序列中的下一个单词。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [32]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


创建输入-目标对以进行下一个单词预测任务的最简单和最直观的方法之一是创建两个变量x和y，其中x包含输入token，y包含目标，即输入向右移动1位的结果。

In [33]:
enc_sample = enc_text[50:]

In [34]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


一个接一个，预测如下：

In [35]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [36]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


在我们将token转换为嵌入之前，还有一个任务要完成：实现一个高效的数据加载器，该加载器遍历输入数据集并将输入和目标作为 PyTorch 张量返回，这些张量可以视为多维数组。

In [37]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.0.1+cpu


具体来说，我们的目标是返回两个张量：一个输入张量，包括 LLM 看到的文本，另一个目标张量，包含 LLM 需要预测的目标。我们使用滑动窗口方法来创建这些输入-目标对，如下图所示：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

创建数据集和数据加载器，从输入文本数据集中提取块

In [38]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [39]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

`stride` 参数定义了滑动窗口的步长，即我们在文本中移动的距离。当`strde`小于`max_length`时候，我们会在文本中创建重叠的滑动窗口，这有助于
- 重叠采样，增加数据量，提高数据利用率
- 自然语言是连续的，如果样本完全独立（无重叠），模型可能无法很好地学习到跨样本的语言特征。重叠采样可以让模型更好地捕捉跨样本的语义信息，提高上下文的学习能力

如果 stride 过小（比如 1），那么会产生大量的样本，数据集规模会大幅增长，导致计算量和存储需求激增。所以 stride 的选择需要权衡计算资源和模型性能

In [44]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [45]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [42]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


一个使用`stride·`等于上下文长度（这里是4）的示例如下：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

我们还可以创建批量输出

注意，我们在这里增加了步长，以便批次之间没有重叠，因为更多的重叠可能会导致过拟合

In [43]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建词嵌入

为 LLM 准备训练集的最后一步是将token ID 转换为嵌入向量，如下图所示。除此之外，需注意的是，我们首先会以随机值初始化这些嵌入权重。这一初始化为 LLM 的学习过程提供了起始点。我们将在第 5 章中优化嵌入权重，作为 LLM 训练的一部分。

对于GPT类大语言模型（LLM）来说，连续向量表示（Embedding）非常重要，原因在于:
- 深度神经网络和连续向量表示：GPT 类模型（以及其他深度神经网络）是基于大量的矩阵运算和数值计算构建的，尤其是神经元之间的连接权重和偏置在训练过程中不断更新。这些运算要求输入的数据是数值形式的向量，因为神经网络只能对数值数据进行有效计算，而无法直接处理原始的离散文字数据（如单词、句子）。因此，我们需要一种方法将单词、短语或整个文档转换为向量表示，这就是嵌入（Embedding）的作用。
- 向量嵌入的作用：连续向量表示不仅让文本数据可以进入神经网络，还帮助模型捕捉和表示文本之间的语义关系。
- 反向传播算法的要求：深度神经网络通过反向传播算法进行训练，反向传播的本质是利用梯度下降法来更新网络的权重，以最小化损失函数（loss function）。反向传播要求每一层的输入、输出和权重都能够参与梯度计算，而梯度计算只能应用于数值数据。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

假设有如下例子：

In [46]:
input_ids = torch.tensor([2, 3, 5, 1])

为了简化，我们假设有一个小词汇表，只有6个单词，我们想要创建大小为3的嵌入向量：

In [47]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [49]:
print(embedding_layer.weight)
print(embedding_layer.weight.size())

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
torch.Size([6, 3])


对于熟悉 one-hot 编码的人来说，上面的嵌入层方法本质上只是实现了一种更高效的方法，即在全连接层中实现 one-hot 编码后的矩阵乘法，这在[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)中的补充代码中有描述

由于嵌入层只是一种更高效的实现，等效于 one-hot 编码和矩阵乘法方法，因此它可以被视为一个可以通过反向传播进行优化的神经网络层

为了将 id 为 3 的 token 转换为一个 3 维向量，我们执行以下操作：

In [ ]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


注意到，上面的操作实际上是在嵌入层权重矩阵中的第4行中查找对应的向量，这个向量就是我们的嵌入向量

为了将所有四个 `input_ids` 值转换为嵌入向量，我们执行以下操作：

In [ ]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


一个嵌入层本质上是一个查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **You may be interested in the bonus content comparing embedding layers with regular linear layers: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 位置编码

嵌入层将 ID 转换为相同的向量表示，而不考虑它们在输入序列中的位置。

从原则上讲，确定性的、与位置无关的token ID 嵌入对于可重复性是有益的。然而，由于LLM的自注意力机制本身也是与位置无关的，因此向LLM注入额外的位置信息是有帮助的。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

有两种主要的位置编码方法：绝对位置编码和相对位置编码。

绝对位置编码与序列中的特定位置直接相关。对于输入序列中的每个位置，都会将一个唯一的绝对位置嵌入向量添加到token的嵌入向量中，以传达其确切位置。例如，第一个token将具有特定的位置嵌入，第二个token将具有另一个不同的嵌入，依此类推，如下图所示。

相对位置编码强调的是token之间的相对位置或距离。这意味着模型学习的是“相隔多远”的关系，而不是“在什么确切位置”。这样的优势在于，即使模型在训练时没有接触过不同的长度，它也可以更好地适应各种长度的序列。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

BytePair 编码器的词汇表大小为 50,257。假设我们想将输入 token 编码为 256 维向量表示：

In [50]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

如果我们从数据加载器中采样数据，我们将每个批次中的 token 嵌入为 256 维向量。如果我们的批次大小为 8，每个批次有 4 个 token，那么结果是一个 8 x 4 x 256 的张量：

In [51]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [52]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [53]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


OpenAI 的 GPT 模型使用绝对位置嵌入，这些嵌入在训练过程中进行优化，而不是像原始 Transformer 模型中的位置编码那样是固定或预定义的。这个优化过程属于模型训练的一部分，我们将在本书后面的章节中实现。目前，让我们创建初始位置嵌入。对于绝对位置编码，只需要创建另一个嵌入层，其维度与 token_embedding_layer 的维度相同：

In [54]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

如前面的代码所示， pos_embeddings 的输入通常是一个占位符向量torch.arange(context_length)，它包含一个从0到最大输入长度-1的数字序列。

context_length 是一个表示LLM支持的输入大小的变量。在这里，我们设置它与输入文本的最大长度相同。在实际应用中，输入文本可能会超过支持的上下文长度，此时我们需要对文本进行截断。

In [55]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


为了创建LLM中使用的输入嵌入，我们只需将token和位置嵌入相加：

- 这里我们使用了广播机制，将两个张量相加，这是PyTorch的一个特性，它允许我们在不同形状的张量之间执行元素级操作，而无需显式地扩展张量的形状。从而避免了需要为每个句子的每个token单独计算位置编码的麻烦，因为每个batch中的所有句子都是相同长度的（假设经过padding）

In [56]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


在输入处理的初始阶段，输入文本被分割为单独的token。接下来，这些token根据预定义的词汇表转换为token ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# Summary and takeaways

See the [./dataloader.ipynb](./dataloader.ipynb) code notebook, which is a concise version of the data loader that we implemented in this chapter and will need for training the GPT model in upcoming chapters.

See [./exercise-solutions.ipynb](./exercise-solutions.ipynb) for the exercise solutions.

See the [Byte Pair Encoding (BPE) Tokenizer From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) notebook if you are interested in learning how the GPT-2 tokenizer can be implemented and trained from scratch.